In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier


In [2]:
iris = load_iris()
X = iris.data[:, 2:]
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [3]:
from sklearn.tree import export_graphviz
export_graphviz(
            tree_clf,
            out_file="./iris_tree.dot",
            feature_names=iris.feature_names[2:],
            class_names=iris.target_names,
            rounded=True,
            filled=True
        )

In [4]:
! dot -Tpng iris_tree.dot -o iris_tree.png

One of the many qualities of Decision Trees is that they require very little data preparation. In particular, they don’t require feature scaling or centering at all.

* Gini impurity:

 $$G_{i}=1-\sum_{k=1}^{n} p_{i, k}^{2}$$

 $p_{i,k}$ is the ratio of class $k$ instances among the training instances in the $i$th node.

Scikit-Learn uses the CART algorithm, which produces only binary trees: nonleaf nodes always have two children
(i.e., questions only have yes/no answers). However, other algorithms such as ID3 can produce Decision Trees with
nodes that have more than two children.

In [6]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

Scikit-Learn uses the Classification And Regression Tree (CART) algorithm to train Decision Trees (also called “growing” trees). The idea is really quite simple: the algo‐ rithm first splits the training set in two subsets using a single feature
$k$ and a threshold $t_k$ (e.g., “petal length ≤ 2.45 cm”). How does
it choose $k$ and $t_k$? It searches for the pair $(k, t_k)$ that
produces the purest subsets (weighted by their size). The cost
function that the algorithm tries to minimize is given by
\begin{align}
\begin{array}{l}
J\left(k, t_{k}\right)=\frac{m_{\text {left }}}{m} G_{\text {left }}+\frac{m_{\text {right }}}{m} G_{\text {right }} \\
\text { where }\left\{\begin{array}{l}
G_{\text {left/right }} \text { measures the impurity of the left/right subset, } \\
m_{\text {left/right }} \text { is the number of instances in the left/right subset. }
\end{array}\right.
\end{array}
\end{align}

Once it has successfully split the training set in two, it splits the
subsets using the same logic, then the sub-subsets and so on, recursively.
It stops recursing once it reaches the maximum
depth (defined by the max_depth hyperparameter), or if it cannot find
a split that will reduce impurity. A few other hyperparameters
(described in a moment) control additional stopping conditions
(min_samples_split, min_sam ples_leaf, min_weight_fraction_leaf,
and max_leaf_nodes).

finding the optimal tree is known to be an NP-Complete problem:2 it requires $O(\exp(m))$ time,
 making the problem intractable even for fairly small training sets. This is why we must settle for a “reasonably good” solution.

#### Gini Impurity or Entropy?

$$H_{i}=-\sum_{k=1 \atop p_{i, k} \neq 0}^{n} p_{i, k} \log _{2}\left(p_{i, k}\right)$$

So should you use Gini impurity or entropy? The truth is, most of the time it does not make a big difference: they lead to similar trees. Gini impurity is slightly faster to compute, so it is a good default. However, when they differ, Gini impurity tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees.


### Regularization Hyperparameters

Such a model is often called a nonparametric model, not because it does not have any parameters (it often has a lot) but because the number of parameters is not determined prior to training, so the model structure is free to stick closely to the data.

In contrast, a parametric model such as a linear model has a predetermined number of parameters, so its degree of freedom is limited, reducing the risk of overfitting (but increasing the risk of underfitting).

The DecisionTreeClassifier class has a few other parameters that similarly restrict the shape of the Decision Tree:
1. min_samples_split (the minimum number of samples a node must have before it can be split)
2. min_samples_leaf (the minimum number of samples a leaf node must have)
3. min_weight_fraction_leaf (same as min_samples_leaf but expressed as a fraction of the total number of weighted
instances)
4. max_leaf_nodes (maximum number of leaf nodes)
5. max_features (maximum number of features that are evaluated for splitting at each node). Increasing min_* hyperparameters or reducing max_* hyperparameters will regularize the model.

Other algorithms work by first training the Decision Tree without restrictions, then pruning (deleting) unnecessary nodes. A node whose children are all leaf nodes is considered unnecessary if the purity improvement it provides is not statistically significant. Stan‐ dard statistical tests, such as the χ2 test, are used to estimate the probability that the improvement is purely the result of chance (which is called the null hypothesis). If this probability, called the p- value, is higher than a given threshold (typically 5%, controlled by a hyperparameter), then the node is considered unnecessary and its children are deleted. The pruning continues until all unnecessary nodes have been pruned.

### Regression

In [8]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [9]:
tree_reg.predict(X)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.09259259, 1.09259259, 1.09259259, 1.09259259, 1.09259259,
       1.09259259, 1.09259259, 1.09259259, 1.09259259, 1.09259259,
       1.09259259, 1.09259259, 1.09259259, 1.09259259, 1.09259259,
       1.09259259, 1.09259259, 1.09259259, 1.09259259, 1.09259259,
       1.97826087, 1.09259259, 1.09259259, 1.09259259, 1.09259

The CART algorithm works mostly the same way as earlier, except that instead of try‐ ing to split the training set in a way that minimizes impurity, it now tries to split the training set in a way that minimizes the MSE. Equation 6-4 shows the cost function that the algorithm tries to minimize.

\begin{align*}
&J\left(k, t_{k}\right)=\frac{m_{\mathrm{left}}}{m} \mathrm{MSE}_{\mathrm{left}}+\frac{m_{\text {right }}}{m} \mathrm{MSE}_{\text {right }}\\
&\text { where }\left\{\begin{array}{l}
\mathrm{MSE}_{\text {node }}=\sum_{i \in \text { node }}\left(\hat{y}_{\text {node }}-y^{(i)}\right)^{2} \\
\hat{y}_{\text {node }}=\frac{1}{m_{\text {node }}} \sum_{i \in \operatorname{node}} y^{(i)}
\end{array}\right.
\end{align*}

### Instability

Decision Trees have a lot going for them: they are simple to understand and interpret, easy to use, versatile, and powerful.
There are few limitations:
* prefer orthogonal decision boundaries, which sensitive to training set rotation.
* More generally, the main issue with Decision Trees is that they are very sensitive to small variations in the training data.


Actually, since the training algorithm used by Scikit-Learn is stochastic6 you may get very different models even on the same training data (unless you set the random_state hyperparameter).